# Compare crop yields to observations

- Uses raw annual CTSM outputs (NOT timeseries files).

Notebook created by Sam Rabin (samrabin@ucar.edu).

In [ ]:
import sys
import os
import glob
import warnings
import numpy as np
import xarray as xr
import pandas as pd
from time import time
from dask.distributed import Client, wait
import convert_pft1d_to_sparse
import importlib
import earthstat
import caselist
import crop_timeseries_figs
import plotting_utils
import bokeh_html_utils

# Plotting utils
import matplotlib.pyplot as plt
import clm_and_earthstat_maps as caem

# Start a local Dask cluster using all available cores
client = Client()
client

## 1. Settings

### 1.1 Parameters modifiable in config.yml

In [ ]:
# Path to CUPiD externals. This method is supposedly unreliable, so it's best for this to be
# overridden by a value given in config.yml. See examples/crops/config.yml.
externals_path = os.path.join(os.getcwd(), os.pardir, os.pardir, "externals")

# Where land output is stored
CESM_output_dir = os.path.join(
    os.path.sep,
    "glade",
    "work",
    "samrabin",
    "clm6_crop_reparam_outputs",
)

# Full casenames that are present in CESM_output_dir and in individual filenames
case_name_list = [
    # "ctsm53019_f09_BNF_hist",
    # "clm6_crop_032",
    # "clm6_crop_032_nomaxlaitrig",
    # "clm6_crop_032_nmlt_phaseparams",
    "alpha-ctsm5.4.CMIP7.09.ctsm5.3.068",
    # "alpha-ctsm5.4.CMIP7.09.ctsm5.3.068_nogddadapt",
    "crujra_matreqs",
    # "crujra_matreqs_nogddadapt",
    # "clm6_crop_032_nmlt_arooti",
    # "crujra_matreqs_and_gdd20blv2",
    # "crujra_matreqs_gdd20blv2_nomxmatv2",
    "clm6_crops_omni01",
    "clm6_crops_omni02",
    "clm6_crops_omni02_unsimgeneric",
]

# Names of cases to show in figure legends
case_legend_list = [
    # "A: ctsm5.3.019 (GSWP3)",
    # "B: ctsm5.3.032 (CRU-JRA)",
    # "C: As B + w/o max LAI triggering grainfill",
    # "D: As C + pre-CLM5 crop phase params",
    "E: 5.4 branch",
    # "F: 5.4 branch (no GDD adapt)",
    "G: CRU-JRA mat reqs",
    # "H: CRU-JRA mat reqs (no GDD adapt)",
    # "I: As C + pre-CLM5 arooti",
    # "J: CRU-JRA mat reqs + gdd20",
    # "K: CRU-JRA mat reqs + gdd20, mxmat360",
    "Omnibus 1",
    "Omnibus 2",
    "Omnibus 2 no CFT merge",
]


# # Where land output is stored
# CESM_output_dir = os.path.join(
#     os.path.sep,
#     "glade",
#     "work",
#     "samrabin",
#     "wwieder_run_outputs",
# )
# # Full casenames that are present in CESM_output_dir and in individual filenames
# case_name_list = [
#     # "ctsm53n04ctsm52028_f09_hist",  # Doesn't have GRAINC_TO_FOOD_PERHARV
#     # "ctsm53041_54surfdata_snowTherm_100_HIST",
#     "ctsm5.4_5.3.068_PPEcal115f09_118_HIST",
#     "ctsm5.4.CMIP7_ciso_ctsm5.3.075_f09_124_HIST",
# ]
# # Names of cases to show in figure legends
# case_legend_list = [
#     # "ctsm53n04ctsm52028_f09_hist",  # Doesn't have GRAINC_TO_FOOD_PERHARV
#     # "Run 100",
#     "Run 118",
#     "Run 124",
# ]


# The actual netCDF timesteps, not the names of the files
start_year = 1961
end_year = 2024

cfts_to_include = [
    "temperate_corn",
    "tropical_corn",
    "cotton",
    "rice",
    "temperate_soybean",
    "tropical_soybean",
    "sugarcane",
    "spring_wheat",
    "irrigated_temperate_corn",
    "irrigated_tropical_corn",
    "irrigated_cotton",
    "irrigated_rice",
    "irrigated_temperate_soybean",
    "irrigated_tropical_soybean",
    "irrigated_sugarcane",
    "irrigated_spring_wheat",
]

crops_to_include = [
    "corn",
    "cotton",
    "rice",
    "soybean",
    "sugarcane",
    "wheat",
]
fao_to_clm_dict = {
    "Maize": "corn",
    "Rice": "rice",
    "Seed cotton, unginned": "cotton",
    "Soya beans": "soybean",
    "Sugar cane": "sugarcane",
    "Wheat": "wheat",
}

verbose = True

obs_data_dir = os.path.join(
    os.sep + "glade",
    "campaign",
    "cesm",
    "development",
    "cross-wg",
    "diagnostic_framework",
    "CUPiD_obs_data",
)

force_new_cft_ds_file = False
force_no_cft_ds_file = False

### 1.2 Other settings

In [ ]:
# Set up directory for any scratch output
if "SCRATCH" in os.environ:
    cupid_temp = os.path.join(os.environ["SCRATCH"], "CUPiD_scratch")
    os.makedirs(cupid_temp, exist_ok=True)
else:
    cupid_temp = "."

N_PFTS = 78

short_names = [case.split(".")[-1] for case in case_name_list]

if start_year > end_year:
    raise RuntimeError(f"start_year ({start_year}) > end_year ({end_year})")

if case_legend_list:
    if len(case_name_list) != len(case_legend_list):
        raise RuntimeError("case_legend_list must be same length as case_name_list")
else:
    case_legend_list = case_name_list

In [ ]:
# Move options to dict for easier passing among functions
opts = {}
opts["CESM_output_dir"] = CESM_output_dir
del CESM_output_dir
opts["case_name_list"] = case_name_list
del case_name_list
opts["case_legend_list"] = case_legend_list
del case_legend_list
opts["start_year"] = start_year
del start_year
opts["end_year"] = end_year
del end_year
opts["cfts_to_include"] = cfts_to_include
del cfts_to_include
opts["crops_to_include"] = crops_to_include
del crops_to_include
opts["fao_to_clm_dict"] = fao_to_clm_dict
del fao_to_clm_dict
opts["verbose"] = verbose
del verbose
opts["obs_data_dir"] = obs_data_dir
del obs_data_dir
opts["force_new_cft_ds_file"] = force_new_cft_ds_file
del force_new_cft_ds_file
opts["force_no_cft_ds_file"] = force_no_cft_ds_file
del force_no_cft_ds_file

### 1.3 Import stuff from externals

In [ ]:
sys.path.append(externals_path)
import ctsm_postprocessing.utils as utils
from ctsm_postprocessing.crops import crop_secondary_variables as c2o
from ctsm_postprocessing.crops import cropcase
import ctsm_postprocessing.crops.faostat as faostat
from ctsm_postprocessing.resolutions import identify_resolution

## 2. Import case data

### 2.1 Import cases

In [ ]:
importlib.reload(cropcase)
importlib.reload(caselist)

case_list = caselist.CaseList(
    CropCase=cropcase.CropCase,
    identify_resolution=identify_resolution,
    opts=opts,
)

### 2.3 Import FAOSTAT

In [ ]:
fao_file = os.path.join(
    opts["obs_data_dir"],
    "lnd",
    "analysis_datasets",
    "ungridded",
    "timeseries",
    "FAOSTAT",
    "Production_Crops_Livestock_2025-02-25",
    "norm",
    "Production_Crops_Livestock_E_All_Data_(Normalized).csv",
)

fao = faostat.FaostatProductionCropsLivestock(
    fao_file,
    y1=opts["start_year"],
    yN=opts["end_year"],
)

# TODO: Move all the following to FaostatProductionCropsLivestock class

fao_prod = fao.get_element("Production", fao_to_clm_dict=opts["fao_to_clm_dict"])
fao_area = fao.get_element("Area harvested", fao_to_clm_dict=opts["fao_to_clm_dict"])

# Only include where both production and area data are present
def drop_a_where_not_in_b(a, b):
    return a.drop([i for i in a.index.difference(b.index)])


fao_prod = drop_a_where_not_in_b(fao_prod, fao_area)
fao_area = drop_a_where_not_in_b(fao_area, fao_prod)
if not fao_prod.index.equals(fao_area.index):
    raise RuntimeError("Mismatch of prod and area indices after trying to align them")

# Don't allow production where no area
is_bad = (fao_prod["Value"] > 0) & (fao_area["Value"] == 0)
where_bad = np.where(is_bad)[0]
bad_prod = fao_prod.iloc[where_bad]
bad_area = fao_area.iloc[where_bad]
fao_prod = fao_prod[~is_bad]
fao_area = fao_area[~is_bad]
if not fao_prod.index.equals(fao_area.index):
    raise RuntimeError(
        "Mismatch of prod and area indices after disallowing production where no area"
    )

# Get yield
fao_yield = fao_prod.copy()
fao_yield["Element"] = "Yield"
fao_yield["Unit"] = "/".join([fao_prod["Unit"].iloc[0], fao_area["Unit"].iloc[0]])
fao_yield["Value"] = fao_prod["Value"] / fao_area["Value"]

# Get dict
fao_dict = {}
fao_dict["yield"] = fao_yield
fao_dict["prod"] = fao_prod
fao_dict["area"] = fao_area

### 2.3 Import EarthStat (basically gridded FAOSTAT)

In [ ]:
importlib.reload(earthstat)

earthstat_dir = os.path.join(
    opts["obs_data_dir"],
    "lnd",
    "analysis_datasets",
    "multi_grid",
    "annual",
    "FAO-EarthStatYields",
)

earthstat_data = earthstat.EarthStat(earthstat_dir, case_list.resolutions, opts)

In [ ]:
importlib.reload(crop_timeseries_figs)
importlib.reload(earthstat)

# Get versions of CLM stats as if planted with EarthStat area
# TODO: Don't hard-code EARTHSTAT_RES_TO_PLOT here
EARTHSTAT_RES_TO_PLOT = "f09"
for case in case_list:
    case_ds = case.cft_ds

    for i, crop in enumerate(opts["crops_to_include"]):
        # Get EarthStat area
        crop_area_es = utils.ungrid(
            gridded_data=earthstat_data[EARTHSTAT_RES_TO_PLOT].get_data("area", crop),
            ungridded_ds=case_ds,
        )

        # Setup crop_*crop_area_es_expanded variable or append to it
        if i == 0:
            crop_area_es_expanded = crop_area_es.expand_dims(dim="crop", axis=0)
        else:
            # Append this crop's DataArray to existing one
            crop_area_es_expanded = xr.concat(
                [crop_area_es_expanded, crop_area_es],
                dim="crop",
            )

    # Convert area units
    clm_units = case_ds["crop_area"].attrs["units"]
    es_units = crop_area_es.attrs["units"]
    if clm_units == "m2" and es_units == "Mha":
        crop_area_es_expanded *= 1e4 * 1e6
        crop_area_es_expanded.attrs["units"] = "m2"
    else:
        raise NotImplementedError(
            f"Conversion assumes CLM area in m2 (got {clm_units}) and EarthStat area in Mha (got {es_units})"
        )

    # Before saving, check alignment of all dims
    crop_area_es_expanded = earthstat.check_dim_alignment(
        crop_area_es_expanded, case_ds
    )

    # Save to case_ds, filling with NaN as necessary (e.g., if there are CLM years not in EarthStat).
    case_ds["crop_area_es"] = crop_area_es_expanded

    # Calculate production as if planted with EarthStat area
    area_units = case_ds["crop_area_es"].attrs["units"]
    area_units_exp = "m2"
    yield_units = case_ds["crop_yield"].attrs["units"]
    yield_units_exp = "g/m2"
    if area_units != area_units_exp or yield_units != yield_units_exp:
        raise NotImplementedError(
            f"Yield calculation assumes area in {area_units_exp} (got {area_units}) and yield in {yield_units_exp} (got {yield_units})"
        )
    case_ds["crop_prod_es"] = case_ds["crop_area_es"] * case_ds["crop_yield"].rename(
        {"pft": "gridcell"}
    )
    case_ds["crop_prod_es"].attrs["units"] = "g"

    # Save EarthStat time axis to avoid plotting years with no EarthStat data
    earthstat_time = crop_area_es_expanded["time"]
    earthstat_time = earthstat_time.rename({"time": "earthstat_time_coord"})
    case_ds["earthstat_time"] = earthstat_time

## 3. Time series figures

The "Area source" menu allows choosing between statistics calculated using different crop areas:
* CLM: Crop areas used in the CLM simulation after all CFT merging has taken place, such as rye being merged to spring wheat.
* EarthStat: Crop yields from the CLM simulation but areas from EarthStat. Note that this will not give you the same results as you would get if you actually ran CLM with the EarthStat areas, because here we are just multiplying CLM's yields by EarthStat areas. If EarthStat has some crop in a gridcell but CLM doesn't, we will get a zero there for our CLM x EarthStat yields and production. Note also that the CLM simulation lines in the area figure might not align perfectly with one another or EarthStat due to differing land masks.

In [ ]:
importlib.reload(crop_timeseries_figs)
importlib.reload(bokeh_html_utils)

# Dictionary whose keys will be used as dropdown menu options and whose values
# will be used for the use_earthstat_area arg in crop_timeseries_figs(). At the
# moment this could work as radio buttons, but I'd like to eventually add a few
# more observational data sources.
area_source_dict = {
    "CLM": False,
    "EarthStat": True,
}

# Dictionary whose keys will be used as radio button options and whose values
# will be used as inputs to crop_timeseries_figs()
stat_dict = {
    "Yield": "yield",
    "Production": "prod",
    "Area": "area",
}

# Where figure files will be saved
img_dir = os.path.join("Global_crop_yield_compare_obs", "timeseries_yieldprodarea")
os.makedirs(img_dir, exist_ok=True)

# Build dropdown specs
dropdown_specs = [
    {
        "title": "Area source",
        "options": list(area_source_dict.keys()),
    }
]

# Build radio specs
radio_specs = [
    {
        "title": "Statistic",
        "options": list(stat_dict.keys()),
    }
]

for stat, stat_input in stat_dict.items():
    for area_source, use_earthstat_area in area_source_dict.items():

        # Get filename to which figure will be saved. Members of join_list
        # must first be any dropdown menu members and then any radio button
        # group members, in the orders given in dropdown_specs and radio_specs,
        # respectively.
        join_list = [area_source, stat]
        fig_basename = bokeh_html_utils.sanitize_filename("_".join(join_list))
        fig_basename += ".png"
        fig_path = os.path.join(img_dir, fig_basename)

        with warnings.catch_warnings():
            # This suppresses some very annoying warnings when
            # use_earthstat_area=True. I'd like to eventually resolve this
            # properly, which will probably requiring compute()ing some of
            # the metadata variables in the cft_ds Datasets.
            warnings.filterwarnings(
                "ignore",
                message="Sending large graph.*",
                category=UserWarning,
            )
            crop_timeseries_figs.main(
                stat_input,
                earthstat_data,
                case_list,
                fao_dict[stat_input],
                opts,
                use_earthstat_area=use_earthstat_area,
                fig_file=fig_path,
            )

# Display in notebook
bokeh_html_utils.create_static_html(
    dropdown_specs=dropdown_specs,
    radio_specs=radio_specs,
    output_dir=img_dir,
    show_in_notebook=True,
)

## 4. Yield maps

In [ ]:
importlib.reload(plotting_utils)
importlib.reload(caem)

caem.clm_and_earthstat_maps(
    which="yield",
    case_list=case_list,
    earthstat_data=earthstat_data,
    utils=utils,
    opts=opts,
)

In [ ]:
importlib.reload(caem)

caem.clm_and_earthstat_maps(
    which="prod",
    case_list=case_list,
    earthstat_data=earthstat_data,
    utils=utils,
    opts=opts,
)

In [ ]:
importlib.reload(caem)

caem.clm_and_earthstat_maps(
    which="area",
    case_list=case_list,
    earthstat_data=earthstat_data,
    utils=utils,
    opts=opts,
)

## 5. Immature and failed harvests

In [ ]:
importlib.reload(plotting_utils)

for crop in opts["crops_to_include"]:
    results = plotting_utils.ResultsMaps(vrange=[0, 1])
    suptitle = None
    for case in case_list:
        tmp = case.cft_ds.sel(crop=crop)
        tmp["frac_immature_harv_timemean"] = tmp["crop_harv_area_immature"].sum(
            dim="time"
        ) / tmp["crop_harv_area"].sum(dim="time")
        map_clm = utils.grid_one_variable(tmp, "frac_immature_harv_timemean")
        map_clm.attrs["units"] = "unitless"
        map_clm.name = "Fraction immature harvests"
        results[case.name] = map_clm
        if suptitle is None:
            suptitle = f"{results[case.name].name}: {crop}"
    results.plot(
        subplot_title_list=case_list.names, suptitle=suptitle, one_colorbar=True
    )

In [ ]:
importlib.reload(plotting_utils)

for crop in opts["crops_to_include"]:
    results = plotting_utils.ResultsMaps(vrange=[0, 1])
    suptitle = None
    for case in case_list:
        tmp = case.cft_ds.sel(crop=crop)
        tmp["frac_failed_harv_timemean"] = tmp["crop_harv_area_failed"].sum(
            dim="time"
        ) / tmp["crop_harv_area"].sum(dim="time")
        map_clm = utils.grid_one_variable(tmp, "frac_failed_harv_timemean")
        map_clm.attrs["units"] = "unitless"
        map_clm.name = "Fraction failed harvests"
        results[case.name] = map_clm
        if suptitle is None:
            suptitle = f"{results[case.name].name}: {crop}"
    results.plot(
        subplot_title_list=case_list.names, suptitle=suptitle, one_colorbar=True
    )

In [ ]:
importlib.reload(crop_timeseries_figs)

# Get figure layout info
fig_opts, fig, axes = crop_timeseries_figs.setup_fig(opts)

for i, crop in enumerate(opts["crops_to_include"]):
    ax = axes.ravel()[i]
    plt.sca(ax)

    # Plot case data
    for c, case in enumerate(case_list):

        crop_data_ts = case.cft_ds.sel(crop=crop)["crop_harv_area_immature"].sum(
            dim=["pft"]
        ) / case.cft_ds.sel(crop=crop)["crop_harv_area"].sum(dim=["pft"])

        # Change line style for one line that overlaps another for some crops
        # TODO: Optionally define linestyle for each case in config.yml
        if "clm6_crop_032_nomaxlaitrig" in opts["case_name_list"] and opts[
            "case_name_list"
        ][c].endswith("clm6_crop_032_nmlt_phaseparams"):
            linestyle = "--"
        else:
            linestyle = "-"

        # Plot
        fig_opts["title"] = "Fraction immature crop area"
        crop_data_ts.plot(linestyle=linestyle)

    # Finish plot
    ax.set_title(crop)
    plt.xlabel("")

crop_timeseries_figs.finish_fig(opts, fig_opts, fig, incl_obs=False)

In [ ]:
importlib.reload(crop_timeseries_figs)

# Get figure layout info
fig_opts, fig, axes = crop_timeseries_figs.setup_fig(opts)

for i, crop in enumerate(opts["crops_to_include"]):
    ax = axes.ravel()[i]
    plt.sca(ax)

    # Plot case data
    for c, case in enumerate(case_list):

        crop_data_ts = case.cft_ds.sel(crop=crop)["crop_harv_area_failed"].sum(
            dim=["pft"]
        ) / case.cft_ds.sel(crop=crop)["crop_harv_area"].sum(dim=["pft"])

        # Change line style for one line that overlaps another for some crops
        # TODO: Optionally define linestyle for each case in config.yml
        if "clm6_crop_032_nomaxlaitrig" in opts["case_name_list"] and opts[
            "case_name_list"
        ][c].endswith("clm6_crop_032_nmlt_phaseparams"):
            linestyle = "--"
        else:
            linestyle = "-"

        # Plot
        fig_opts["title"] = "Fraction failed crop area"
        crop_data_ts.plot(linestyle=linestyle)

    # Finish plot
    ax.set_title(crop)
    plt.xlabel("")

crop_timeseries_figs.finish_fig(opts, fig_opts, fig, incl_obs=False)

## 6. Growing seasons

### 6.1 Overwintering in CLM

In [ ]:
# Calculate overwintering, if needed or requested
force_recalc = False

from ctsm_postprocessing.crops import combine_cft_to_crop

importlib.reload(combine_cft_to_crop)

for case in case_list:
    ds = case.cft_ds

    if (
        "overwinter_frac_crop_timemean" in ds
        and "overwinter_area_crop" in ds
        and not force_recalc
    ):
        continue
    nwifiern

    is_nh = ds["pfts1d_lat"] >= 0
    nh_overwinter = is_nh & (ds["HDATES"] < ds["SDATES_PERHARV"])
    sh_overwinter = ~is_nh & (ds["SDATES_PERHARV"] < 182.5) & (ds["HDATES"] > 182.5)
    overwinter = (nh_overwinter | sh_overwinter) & (ds["HARVEST_REASON_PERHARV"] > 0)

    ds["overwinter_area"] = (overwinter * ds["cft_harv_area"]).sum(dim="mxharvests")
    ds = combine_cft_to_crop.combine_cft_to_crop(
        ds, "overwinter_area", "overwinter_area_crop", method="sum"
    )

    # This should be changed to happen automatically elsewhere!
    ds["overwinter_area_crop"].attrs["units"] = "m2"

    ds["overwinter_frac_crop_timemean"] = ds["overwinter_area_crop"].sum(
        dim="time"
    ) / ds["crop_harv_area"].sum(dim="time")
    assert not np.any(ds["overwinter_frac_crop_timemean"] < 0)
    assert not np.any(ds["overwinter_frac_crop_timemean"] > 1)

    # This should be changed to happen automatically elsewhere!
    ds["overwinter_frac_crop_timemean"].attrs["units"] = "unitless"

    # Mask
    ds["overwinter_area_crop"] = ds["overwinter_area_crop"].where(
        ds["crop_harv_area"].sum(dim="time") > 0
    )

importlib.reload(plotting_utils)
importlib.reload(utils)

for crop in opts["crops_to_include"]:
    results_area = plotting_utils.ResultsMaps()
    results_frac = plotting_utils.ResultsMaps(vrange=[0, 1])

    suptitle_area = None
    suptitle_frac = None
    for case in case_list:
        ds = case.cft_ds.sel(crop=crop)

        results_area[case.name] = utils.grid_one_variable(
            ds.mean(dim="time", keep_attrs=True), "overwinter_area_crop"
        )
        results_area[case.name].name = "Overwintering area"

        results_frac[case.name] = utils.grid_one_variable(
            ds, "overwinter_frac_crop_timemean"
        )
        results_frac[case.name].name = "Overwintering fraction"

        if suptitle_area is None:
            suptitle_area = f"{results_area[case.name].name}: {crop}"
        if suptitle_frac is None:
            suptitle_frac = f"{results_area[case.name].name}: {crop}"
    results_area.plot(
        subplot_title_list=case_list.names, suptitle=suptitle_area, one_colorbar=True
    )
    results_frac.plot(
        subplot_title_list=case_list.names, suptitle=suptitle_frac, one_colorbar=True
    )

### 6.2 Overwintering in GGCMI seasons ("observations")

In [ ]:
importlib.reload(plotting_utils)
importlib.reload(bokeh_html_utils)

subplot_title_list = [
    "Sowing date",
    "Harvest date",
    "Growing season length",
    "Overwinter?",
    "Data source",
]

crop_cal_dir = os.path.join(
    opts["obs_data_dir"],
    "lnd",
    "analysis_datasets",
    "ggcmi_grid",
    "annual_avg",
    "crop_calendar",
)

img_dir = os.path.join("Global_crop_yield_compare_obs", "ggcmi_calendars")
os.makedirs(img_dir, exist_ok=True)

ggcmi_crop_dict = {
    "Corn": "mai",
    "Cotton": "cot",
    "Rice": "ri1",
    "Soy": "soy",
    "Sugarcane": "sgc",
    "Spring wheat": "swh",
}
ggcmi_rfir_dict = {
    "Rainfed": "rf",
    "Irrigated": "ir",
}

for crop, crop_ggcmi in ggcmi_crop_dict.items():
    overwinter = None
    for rfir, rfir_ggcmi in ggcmi_rfir_dict.items():
        results = plotting_utils.ResultsMaps()

        cropi = f"{crop_ggcmi}_{rfir_ggcmi}"
        suptitle = f"GGCMI growing seasons: {cropi}"
        file = os.path.join(
            crop_cal_dir, f"{cropi}_ggcmi_crop_calendar_phase3_v1.01.nc4"
        )
        ds = xr.open_dataset(file, decode_times=False)
        sdates = ds["planting_day"]
        hdates = ds["maturity_day"]

        results["Sowing date"] = sdates
        results.plot_vranges["Sowing date"] = [0, 365]

        results["Harvest date"] = hdates
        results.plot_vranges["Harvest date"] = [0, 365]

        results["Growing season length"] = ds["growing_season_length"]
        results.plot_vranges["Growing season length"] = [0, 365]

        # results["Data source"] = ds["data_source_used"]

        is_nh = ds["lat"] >= 0
        nh_overwinter = is_nh & (hdates < sdates)
        sh_overwinter = ~is_nh & (sdates < 182.5) & (hdates > 182.5)
        overwinter = nh_overwinter | sh_overwinter
        overwinter = overwinter.where(~np.isnan(hdates))
        results["Overwinter?"] = overwinter
        results.plot_vranges["Overwinter"] = [0, 1]

        fig_basename = (
            bokeh_html_utils.sanitize_filename("_".join([crop, rfir])) + ".png"
        )
        fig_path = os.path.join(img_dir, fig_basename)
        results.plot(
            subplot_title_list=subplot_title_list, suptitle=suptitle, fig_path=fig_path
        )

# Build dropdown specs
dropdown_specs = [
    {
        "title": "Crop",
        "options": list(ggcmi_crop_dict.keys()),
    }
]

# Build radio specs
radio_specs = [
    {
        "title": "Irrigated?",
        "options": list(ggcmi_rfir_dict.keys()),
    }
]

importlib.reload(bokeh_html_utils)

# Display in notebook (no HTML file created)
bokeh_html_utils.create_static_html(
    dropdown_specs=dropdown_specs,
    radio_specs=radio_specs,
    output_dir=img_dir,
    show_in_notebook=True,
)
print("Done")

In [ ]:
from ctsm_postprocessing import extending_xarray_ops
from ctsm_postprocessing.timing import Timing

importlib.reload(extending_xarray_ops)
importlib.reload(plotting_utils)

results_clm = plotting_utils.ResultsMaps()
t = Timing()
for case in case_list:
    da = extending_xarray_ops.da_circmean_doy(
        case.cft_ds["SDATES"].isel(mxsowings=0), dim="time"
    )
    results_clm[case.name] = da
t.end_all("Loop")